In [1]:
!pip install spotipy

     |████████████████████████████████| 138 kB 11.9 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '8bc466d7fa994c40b1c0ad5ecde205ad' 
secret = 'b045478038744d0a9a1917e2be70e15d'


client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
spotify_api = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

print(spotify_api)

In [3]:
import json
import pandas as pd

playlist_features_list = ["artist", "album", "track_name", "track_id", "release_year", "explicit", "popularity",
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
                             
def analyze_playlist (username, playlist_id, playlist_year):

  offset = 0
  playlist_df = pd.DataFrame(columns = playlist_features_list)
  playlist = spotify_api.user_playlist_tracks(username, playlist_id, limit = 100, offset=offset)["items"]
  count = 0

  #print(playlist[0]["track"]["album"]["artists"][0]["name"])

  for track in playlist:
    playlist_data = {}
    playlist_data["artist"] = track["track"]["album"]["artists"][0]["name"]
    playlist_data["album"] = track["track"]["album"]["name"]
    playlist_data["track_name"] = track["track"]["name"]
    playlist_data["track_id"] = track["track"]["id"]
    playlist_data["release_year"] = playlist_year
    playlist_data["explicit"] = track["track"]["explicit"]
    playlist_data["popularity"] = track["track"]["popularity"]
    
    audio_data = spotify_api.audio_features(playlist_data["track_id"])[0]
    for data in playlist_features_list[7:]:
      playlist_data[data] = audio_data[data]
    
    track_df = pd.DataFrame(playlist_data, index = [0])
    playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    count += 1
  
  #topRange * 100 is the amount of songs that will be scraped per playlist
  topRange = 100
  for i in range(1,topRange):
    offset = 100 * i
    playlist = spotify_api.user_playlist_tracks(username, playlist_id, limit = 100, offset=offset)["items"]
    #print(playlist[0]["track"]["album"]["artists"][0]["name"] + " " + str(count))
    
    for track in playlist:
      if(track["track"] is not None):
        try:
          playlist_data = {}
          playlist_data["artist"] = track["track"]["album"]["artists"][0]["name"]
          playlist_data["album"] = track["track"]["album"]["name"]
          playlist_data["track_name"] = track["track"]["name"]
          playlist_data["track_id"] = track["track"]["id"]
          playlist_data["release_year"] = playlist_year
          playlist_data["explicit"] = track["track"]["explicit"]
          playlist_data["popularity"] = track["track"]["popularity"]
    
          audio_data = spotify_api.audio_features(playlist_data["track_id"])[0]
          for data in playlist_features_list[7:]:
            playlist_data[data] = audio_data[data]
    
          track_df = pd.DataFrame(playlist_data, index = [0])
          playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
          count += 1
        except:
          print("     exception")
          #print(json.dumps(track, indent=4))

  return list(playlist_df.T.to_dict().values())

In [4]:
def analyze_playlist_dict(playlist_dict):
  playlists = {}
  for i, (key, val) in enumerate(playlist_dict.items()):
    playlist = analyze_playlist(*val)
    print(len(playlist))
    playlists[key] = playlist
  return playlists

In [5]:
playlist_dict = {
    'Top Tracks of 2018' : ("Spotify" , "37i9dQZF1DX1HUbZS4LEyL", 2018),
    'Top Tracks of 2019' : ("Spotify" , "37i9dQZF1DX7fxmJCMXN72", 2019),
    'Top Tracks of 2020' : ("Spotify" , "37i9dQZF1DX7Jl5KP2eZaS", 2020),
    'Top Tracks of 2021' : ("Spotify" , "37i9dQZF1DX18jTM2l2fJY", 2020),
}

In [6]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [7]:
import csv

data = analyze_playlist_dict(playlist_dict)
data_file = open('/drive/My Drive/Data Vis Final/data_file.csv', 'w')
csv_writer = csv.DictWriter(data_file, fieldnames=playlist_features_list)
csv_writer.writeheader()
 
count = 0
for playlist in playlist_dict:
    plData = data[playlist]
    for track in plData:
      csv_writer.writerow(track)
    count += 1
 
data_file.close()

100
50
50
50


In [8]:
spotify_df = pd.read_csv('/drive/My Drive/Data Vis Final/data_file.csv')
print(spotify_df)

              artist                      album  ... duration_ms time_signature
0              Drake                   Scorpion  ...      198973              4
1       XXXTENTACION                          ?  ...      166606              4
2        Post Malone       beerbongs & bentleys  ...      218147              4
3        Post Malone       beerbongs & bentleys  ...      221440              4
4              Drake                   Scorpion  ...      217925              4
..               ...                        ...  ...         ...            ...
245         J Balvin                       JOSE  ...      217360              4
246       Joel Corry  Head & Heart (feat. MNEK)  ...      166028              4
247    Ariana Grande                  Positions  ...      173711              4
248       Tate McRae         you broke me first  ...      169266              4
249  Sebastian Yatra             Pareja Del Año  ...      195053              4

[250 rows x 19 columns]


In [11]:
#Code Provided by Alvin Lee (See https://github.com/MFaria27/DS3010-Spotify-Project/blob/main/DS3010_Project.ipynb for details)

pd.options.mode.chained_assignment = None

#iterate through key column to replace key values with string
for i in range(0, len(spotify_df.key)):
    if spotify_df.key[i] == "C":
        spotify_df.key[i] = 0
    elif spotify_df.key[i] == "C#/Db":
        spotify_df.key[i] = 1
    elif spotify_df.key[i] == "D":
        spotify_df.key[i] = 2
    elif spotify_df.key[i] == "D#/Eb":
        spotify_df.key[i] = 3
    elif spotify_df.key[i] == "E":
        spotify_df.key[i] = 4
    elif spotify_df.key[i] == "F":
        spotify_df.key[i] = 5
    elif spotify_df.key[i] == "F#/Gb":
        spotify_df.key[i] = 6
    elif spotify_df.key[i] == "G":
        spotify_df.key[i] = 7
    elif spotify_df.key[i] == "G#/Ab":
        spotify_df.key[i] = 8
    elif spotify_df.key[i] == "A":
        spotify_df.key[i] = 9
    elif spotify_df.key[i] == "A#/Bb":
        spotify_df.key[i] = 10
    elif spotify_df.key[i] == "B":
        spotify_df.key[i] = 11

#iterate through mode column to replace 0 or 1 with Minor/Major
for i in range(0, len(spotify_df['mode'])):
    if spotify_df['mode'][i] == 'Minor':
        spotify_df['mode'][i] = 0
    elif spotify_df['mode'][i] == 'Major':
        spotify_df['mode'][i] = 1
#print(spotify_df.key.head())

for i in range(0, len(spotify_df['explicit'])):
    if spotify_df.explicit[i] == True:
        spotify_df.explicit[i] = 1
    elif spotify_df.explicit[i] == False:
        spotify_df.explicit[i] = 0

#check correct transformation
print(spotify_df.explicit.head())
print(spotify_df.explicit.tail())

spotify_df.to_csv('/drive/My Drive/Data Vis Final/clean_data_file.csv')

0    1
1    1
2    1
3    1
4    1
Name: explicit, dtype: object
245    0
246    0
247    1
248    0
249    0
Name: explicit, dtype: object


In [12]:
clean_df = pd.read_csv('/drive/My Drive/Data Vis Final/clean_data_file.csv')
print(clean_df)

     Unnamed: 0           artist  ... duration_ms time_signature
0             0            Drake  ...      198973              4
1             1     XXXTENTACION  ...      166606              4
2             2      Post Malone  ...      218147              4
3             3      Post Malone  ...      221440              4
4             4            Drake  ...      217925              4
..          ...              ...  ...         ...            ...
245         245         J Balvin  ...      217360              4
246         246       Joel Corry  ...      166028              4
247         247    Ariana Grande  ...      173711              4
248         248       Tate McRae  ...      169266              4
249         249  Sebastian Yatra  ...      195053              4

[250 rows x 20 columns]
